# Imports

In [1]:
import zipfile
import json
from scipy import sparse
import numpy as np
import time


# Dataset path

In [2]:
datadirectory="/home/leboudy/Downloads/lastfm_train.zip"
dataset = zipfile.ZipFile(datadirectory)

# global variables initialization


In [3]:
songId=0
t=0
g=50
beta=0.2
categories =["Technical Death Metal"]
S =0#[]
#id2songJsonMap = []
id2songIdMap = {}
fhat=[]
n=5 #top songs to report
#limitingCounter = 1000


# Loading data


In [4]:
start_time = time.time()
for i in dataset.namelist():
    if  i.endswith(".json"): 
        f = json.loads(dataset.read(i).decode("utf-8"))
        if  f["track_id"] not in id2songIdMap.keys():
            id2songIdMap[f["track_id"]] = (songId,i)
            tempcategories = set(categories)#will keep removing tags from it
            newtags=[]
            for tag in f["tags"]:#looping over tags
                if int(tag[1]) >g:#filters out those <g
                    newtags.append(tag)#keeping >g
                    if tag[0] in tempcategories:#if it is in the tempcategories
                        tempcategories.remove(tag[0])#removing it
            #f["tags"] = newtags
            #id2songJsonMap.append(f)
            #songTags= [t[0] for t in f["tags"]]
            #if len(list(filter(lambda x: x in categories,songTags )) ) == len(categories):
            if len(tempcategories)==0:#meaning we encountered all of the tags
                S+=1#.append(songId)
                fhat.append(1)
            else:
                fhat.append(0)
            songId+=1 
            
print("--- %s seconds ---" % (time.time() - start_time))

--- 264.47087717056274 seconds ---


# initializing the teleportation probablilities


In [5]:
#_fhat=np.array()
#_fhat.shape=(songId,1)
#fhat=(1/len(S))*np.array(fhat)
fhat=(1/S)*np.array(fhat)

# Building the glorious Adjacency matrix


In [6]:
start_time = time.time()
col=[]
row=[]
data=[]
for i in id2songIdMap.keys():
    f = json.loads(dataset.read(id2songIdMap[i][1]).decode("utf-8"))
    for s in f["similars"]:
        if s[0] in id2songIdMap.keys() and s[1]>=t:
            col.append(id2songIdMap[i][0])
            row.append(id2songIdMap[s[0]][0])
            data.append(1)
    
    
    

        
print("--- %s seconds ---" % (time.time() - start_time))        


--- 185.57483077049255 seconds ---


In [7]:
A = sparse.csr_matrix((data, (row, col)),shape=(songId, songId))

# Randomly initializing r


In [8]:
r =np.random.rand(A.shape[0])
r=r/np.sum(r)

In [9]:
#print(len(id2songJsonMap))
print (A.shape)
print(songId)
print (S)
print (songId)

(839122, 839122)
839122
494
839122


# Power Iterations


In [10]:
for i in range(100):
    r=A.multiply(beta).dot(r)+fhat*(1-beta)

# Reporting the top n songs


In [11]:
topIndecs = r.argsort()[-n:][::-1]


In [12]:
keys= list(id2songIdMap.keys())
for item in topIndecs:
    print (keys[item])

TRZEXMX128F932F964
TRBBFNF12903CEA3F8
TRZQXLA12903D0314B
TROEVAE12903CD9745
TRSAJOV128F4246984


In [13]:
print (topIndecs)

[348755 382258 522309 257574 394833]
